## 상품정보 데이터

### json데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np

import re
import warnings
import os
warnings.filterwarnings(action='ignore')
#pd.set_option('옵션 이름', 수정할 값)
# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)

## items

In [ ]:
items = pd.read_json('./items.json')
items.head()

In [ ]:
items.info()

### 상품명 전처리

In [ ]:
def item_name(row):
    return row.split('\n')[0] # 줄바꿈이전 문자열만 남기기

items['name'] = items['name'].apply(item_name)
items.head()

### 상품명 중복확인

In [ ]:
dup_idx = items.loc[items['name'].duplicated()==True].index
items.drop(dup_idx, axis=0, inplace=True)
items['name'].duplicated().sum()


In [ ]:
# 인덱스 재설정
items.reset_index(drop=True, inplace=True)

In [ ]:
items.info()

### Item_id 부여

In [ ]:
items['item_id'] = [(5-len((str(i)))) * '0' + str(i) for i in list(items.index)]
items.head()

In [ ]:
items.tail()

In [ ]:
# items.to_json('../Load/items_prc.json', orient='records', indent=2, force_ascii=False) # 객체기준 레코드, 들여쓰기 적용, 한글깨짐 해결

In [ ]:
# # 리뷰 크롤링용 데이터 저장
# items[['item_id', 'link']].to_json('../Extract/items_link.json', orient='records', indent=2, force_ascii=False)

# review data

In [61]:
review = pd.read_json('./reviews.json')

# 불필요한 컬럼 삭제
review = review[['품번', '아이디', '별점', '키', '몸무게', '평소사이즈', '타입', '리뷰', '사이즈', '컬러', '색상', '색상-사이즈', '타입2', '데님']]
review.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80123 entries, 0 to 80122
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   품번      80123 non-null  int64 
 1   아이디     80123 non-null  object
 2   별점      80123 non-null  int64 
 3   키       61706 non-null  object
 4   몸무게     56487 non-null  object
 5   평소사이즈   57987 non-null  object
 6   타입      77544 non-null  object
 7   리뷰      80123 non-null  object
 8   사이즈     607 non-null    object
 9   컬러      424 non-null    object
 10  색상      3002 non-null   object
 11  색상-사이즈  527 non-null    object
 12  타입2     47 non-null     object
 13  데님      820 non-null    object
dtypes: int64(2), object(12)
memory usage: 8.6+ MB


### 카디널리티 확인

In [ ]:
for col in ['별점', '키', '몸무게', '평소사이즈']:
    print(col)
    print(review[col].unique())
    print('\n')

* 핫핑 사이즈 기준

<img src='./imgs/사이즈통일.png'>

In [ ]:
# 평소 사이즈 수정
key1 = [str(i) for i in range(44, 100, 11)] +['100']
key2 = ['S', 'M', 'L', 'XL', 'XXL', 'XXXL']

def user_size(row):
    for i in range(1,6):
        if row == key1[i]:
            return str(i)
        elif row == key2[i]:
            return str(i)
        
review['평소사이즈'] = review['평소사이즈'].apply(user_size)
review['평소사이즈'].unique()

### ['사이즈', '컬러', '색상', '타입', '타입2', '색상-사이즈', '데님'] 컬럼 통일 필요 = **['사이즈: str', '색상: 한글']으로 통일**


<img src='./imgs/타입_해당컬럼.png' width=500 height=500/>

#### 각 컬럼별 카디널리티 줄이고 컬럼 통일

In [ ]:
# # 카디널리티 확인
# cols = ['사이즈', '컬러', '색상', '타입', '타입2', '색상-사이즈', '데님']
# for col in cols:
#     print(col)
#     print(review[col].unique())
#     print('\n')

In [ ]:
review = review.replace({np.nan: None})

#### 사이즈 있는경우 '색상-사이즈', 없는 경우 '색상' 형태로 값 통일

In [ ]:
# 사이즈 컬럼
def size(row):
    if row in ['1', '2', '3', '4', 'F', '블랙(black)-2', '블랙 1']:
        return row
    elif row in ['1size', '미니-1', '1사이즈']:
        return re.sub('[^0-9]', '', row)
    elif row=='free':
        return 'F'
review['사이즈'] = review['사이즈'].apply(size)

review.loc[review['사이즈']=='블랙 1', '사이즈'] = '블랙-1'

# 컬러 컬럼
def color(row):
    if row: # Nan => None
        if '_' in row:
            return row.split('_')[-1]
        elif '/' in row:
            return row.split('/')[-1]
        elif '미니-' in row:
            row = row.replace('미니-', '')
            return row
        elif ' ' in row and '(' not in row:
            return row.split()[-1]
        else:
            return row
    
review['컬러'] = review['컬러'].apply(color)


# 타입 컬럼
def type_prc(row):
    if not row: # None값 제외
        pass
    elif '. ' in row: 
        return row.split('-')[-1]
    elif ' [예약판매]' in row:
        row = row.replace(' [예약판매]', '')
        return row
    elif '(반팔)' in row:
        row = row.replace('(반팔)', '')
        return row
    elif '_' in row:
        return row.split('_')[-1]
    elif row[0].isdigit() and ' ' not in row: # return T/F
        return None
    elif row[0].isdigit() and ' ' in row:
        return row.split('-')[-1]
    elif '(' not in row: # 타입과 무관한값은 ()문자가 없음
        return None
    else:
        return row

review['타입'] = review['타입'].apply(type_prc)
    
# 데님, 색상-사이즈 컬럼
def denim_color(row):
    if type(row) == str:
        if '예약' in row:
            return row.split(' ')[0]
        else:
            return row

review['데님'] = review['데님'].apply(denim_color)
review['색상-사이즈'] = review['색상-사이즈'].apply(denim_color)
review['타입2'] = review['타입2'].apply(denim_color)

In [ ]:
# 카디널리티 확인
cols = ['사이즈', '컬러', '색상', '타입', '타입2', '색상-사이즈', '데님']
for col in cols:
    print(col)
    print(review[col].unique())
    print('\n')

In [ ]:
# 색상-사이즈 형태인 경우
    ## str.split()로 분리, 문자열 정규화로 데이터 수정
# 색상 데이터만 있는 경우
    ## 문자열 정규화만 수행
def col_merge(df, cols):
    def normalize(row):
        row[0] = re.sub('[^가-힣]','',row[0]) # 한글만 남기기
        return row

    # 사이즈 컬럼 별도 수행
    idx = df.loc[(review['사이즈']=='블랙-1') | (review['사이즈']=='블랙(black)-2')].index
    for i in idx:
        df.loc[i, ['색상', '사이즈']] = df.iloc[i]['사이즈'].split('-')
    for col in cols:
        idx = df.loc[df[col].notnull()==True].index
        for i in idx:
            row = df.iloc[i][col].split('-')
            if len(row)==2:
                df.loc[i, ['색상', '사이즈']] = normalize(row)
            else:
                df.loc[i, ['색상']] = normalize(row)[0]
    return df
cols = ['컬러', '타입', '타입2', '색상-사이즈', '데님', '색상']
review = col_merge(review, cols)
# review.sample(10)

* 카디널리티 확인

In [ ]:
cols = ['사이즈', '색상']
for col in cols:
    print(col)
    print(review[col].unique())
    print('\n')

In [ ]:
# 현재까지 처리된 df 사본
review_prc = review.copy()

#### 사이즈 컬럼 추가 전처리

In [ ]:
def last_size(row):
    if row:
        if row == 'L':
            return '3'
        elif row == 'B)':
            return None
        elif row == ' F':
            return 'F'
        else:
            return row
review_prc['사이즈'] = review_prc['사이즈'].apply(last_size)

review_prc['사이즈'].unique()

            

* 나머지 컬럼은 삭제

In [ ]:
# 통합 끝난 컬럼 제거 (색상, 사이즈)
review_prc.drop(columns=['컬러', '타입', '타입2', '색상-사이즈', '데님'], inplace=True)

### '별점' 컬럼 핫핑 직원 데이터는 제외

In [ ]:
staffs = set()
p = re.compile('[ㄱ-힣]')
for name in review_prc['아이디'].unique():
    r = p.search(name)
    if r:
        staffs.add(name)
        
for s in staffs:
    review_prc.loc[review_prc['아이디'] == s, '별점'] = -1 # 분석에 사용하지 않음
          

In [60]:
review_prc['별점'].value_counts()

별점
 5    72123
 4     4291
 3     1725
-1      869
 2      565
 1      550
Name: count, dtype: int64

In [ ]:
review_prc.to_json('../Load/reviews_prc_2.json', orient='records', indent=2, force_ascii=False)